# SQLAlchemy Core: Part 1

## Overall Architecture

- The overall architecture is as follows:

**Frontend → API (e.g., Flask or FastAPI) → Backend (e.g., a server running Python) → SQLAlchemy Core (Reads) and SQLAlchemy ORM (Writes) → Database**

- According to [the SQLAlchemy documentation](https://docs.sqlalchemy.org) (2025):
  - **SQLAlchemy Core** is the foundational architecture for SQLAlchemy as a “database toolkit.” The library provides tools for managing connectivity to a database, interacting with database queries and results, and programmatic construction of SQL statements.
  - **SQLAlchemy ORM** builds upon SQLAlchemy Core to provide optional Object Relational Mapping (ORM) capabilities. This includes an additional configuration layer allowing user-defined Python classes to be mapped to database tables and other constructs, as well as an object persistence mechanism known as the Session. It then extends the Core-level SQL Expression Language to allow SQL queries to be composed and invoked in terms of user-defined objects.

## Install SQLAlchemy

- We need to install SQLAlchemy to work with databases in Python.

In [41]:
!pip install sqlalchemy

## Import SQLAlchemy

- We then import the SQLAlchemy library into the Python environment.
- Link to SQLAlchemy documentation: [https://docs.sqlalchemy.org](https://docs.sqlalchemy.org)

In [42]:
import sqlalchemy

The library contains several packages. An alternative way of importing the library is to import specific packages as needed:

In [43]:
from sqlalchemy import exc

Another alternative is to import all packages from SQLAlchemy:

In [44]:
from sqlalchemy import *

## Check the Version of SQLAlchemy

- We can confirm the version of SQLAlchemy installed using the `__version__` attribute. This notebook is based on version `2.0.44` of SQLAlchemy.

In [45]:
sqlalchemy.__version__

'2.0.44'

## Create an Engine to Connect to the Database

- `create_engine()` is a function in SQLAlchemy that creates a new SQLAlchemy Engine instance. An Engine is the starting point for any SQLAlchemy application that wants to interact with a database. It manages the connection pool and provides a source of database connections.
- `echo=True` instructs SQLAlchemy to log all the SQL statements it executes to the console. This is useful for debugging and understanding what SQLAlchemy is doing for you behind the scenes.

In [46]:
engine = create_engine('sqlite+pysqlite:///mydatabase.db', echo=True)

## Connect to the Database using DataGrip

- Connect to the SQLite database using DataGrip (or any other similar software) to verify that the database file `mydatabase.db` has been created.
- In DataGrip, create a new data source and select SQLite. Then, select the `mydatabase.db` file to connect to the database.
- `mydatabase.db` should have been created in the current working directory when you executed the `create_engine()` function.

![img.png](assets/images/1_DataGripTOSQLLite.png)

## Connect to the Database using the engine in Python

In [47]:
conn = engine.connect()

## Execute Raw SQL Statements using SQLAlchemy Core

- We can also execute raw SQL statements using the connection object.
- Each SQL statement should be wrapped in the `text()` function from SQLAlchemy.
- The `commit()` method then commits any changes to the database.

In [48]:
conn.execute(text('CREATE TABLE IF NOT EXISTS supplier (name VARCHAR(255), rating int)'))

conn.commit()

2025-12-08 09:15:44,360 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-08 09:15:44,368 INFO sqlalchemy.engine.Engine CREATE TABLE IF NOT EXISTS supplier (name VARCHAR(255), rating int)
2025-12-08 09:15:44,370 INFO sqlalchemy.engine.Engine [generated in 0.02364s] ()
2025-12-08 09:15:44,393 INFO sqlalchemy.engine.Engine COMMIT


- Confirm that the `supplier` table has been created in the `mydatabase.db` database using DataGrip or any other similar software.

![img.png](assets/images/2_CreateTableConfirmation.png)

In [49]:
conn.execute(text('INSERT INTO supplier (name, rating) VALUES ("Ruiru II Farm", 3);'))

conn.commit()

2025-12-08 09:18:23,598 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-08 09:18:23,606 INFO sqlalchemy.engine.Engine INSERT INTO supplier (name, rating) VALUES ("Ruiru II Farm", 3);
2025-12-08 09:18:23,609 INFO sqlalchemy.engine.Engine [generated in 0.02185s] ()
2025-12-08 09:18:23,619 INFO sqlalchemy.engine.Engine COMMIT


- Confirm that the new record has been inserted.

![img.png](assets/images/3_Insert_Confirmation.png)

- It is recommended to limit the use of the connection object to a specific context. This can be done using a `with` statement, for example:

In [50]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM supplier"))
    print(result.all())

2025-12-08 09:19:37,545 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-08 09:19:37,546 INFO sqlalchemy.engine.Engine SELECT * FROM supplier
2025-12-08 09:19:37,547 INFO sqlalchemy.engine.Engine [generated in 0.00218s] ()
[('Ruiru II Farm', 3)]
2025-12-08 09:19:37,583 INFO sqlalchemy.engine.Engine ROLLBACK


### Create
- Textual SQL is not the usual way we work with SQLAlchemy. However, when using textual SQL, a Python literal value, even non-strings like integers or dates, **should never be 'stringified' directly into an SQL string**; a parameter should always be used. This avoids SQL injection attacks.

In [51]:
with engine.connect() as conn:
    conn.execute(
        text("INSERT INTO supplier (name, rating) VALUES (:name, :rating)"),
        [{"name": "Narok 41 Ranch", "rating": 4.8}, {"name": "Ngong Green House", "rating": 2}],
    )
    conn.commit()

2025-12-08 09:20:33,429 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-08 09:20:33,431 INFO sqlalchemy.engine.Engine INSERT INTO supplier (name, rating) VALUES (?, ?)
2025-12-08 09:20:33,433 INFO sqlalchemy.engine.Engine [generated in 0.00544s] [('Narok 41 Ranch', 4.8), ('Ngong Green House', 2)]
2025-12-08 09:20:33,437 INFO sqlalchemy.engine.Engine COMMIT


- The `BEGIN (implicit)` in the SQLAlchemy Core console means that SQLAlchemy has automatically started a new database transaction for you, without you explicitly calling `begin()`.

- This usually happens when you execute a statement that modifies the database (like `INSERT`, `UPDATE`, or `DELETE`) and no transaction is currently active. The transaction will remain open until you explicitly call `commit()` or `rollback()`.

- Alternatively, we can use the `begin()` method to handle transactions.

  - `engine.begin()` creates a connection and starts a transaction automatically. The transaction is committed if no exceptions occur, or rolled back if an exception is raised. This is useful for ensuring atomic operations and cleaner transaction management.
  - `engine.connect()` creates a database connection that you can use to execute SQL statements. You must manually manage transactions (i.e., call `commit()` or `rollback()` as needed).

- Using `engine.begin()` is generally preferred for transaction management, as it reduces the risk of leaving transactions open unintentionally. However, if you need more control over the connection and transaction lifecycle, you can choose to use `engine.connect()`. This gives you access to the rollback() method if needed.

- Analogy: Driving a car with an automatic transmission (using `begin()`) vs. a manual transmission (using `connect()`).

![img.png](assets/images/4_Automatic-vs-Manual-Transmission.png)

In [52]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO supplier (name, rating) VALUES (:name, :rating)"),
        [{"name": "Makueni Fruits", "rating": 4}, {"name": "Pwani Farm", "rating": 1.8}],
    )
    conn.commit()

2025-12-08 09:24:58,450 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-08 09:24:58,510 INFO sqlalchemy.engine.Engine INSERT INTO supplier (name, rating) VALUES (?, ?)
2025-12-08 09:24:58,514 INFO sqlalchemy.engine.Engine [cached since 265.1s ago] [('Makueni Fruits', 4), ('Pwani Farm', 1.8)]
2025-12-08 09:24:58,529 INFO sqlalchemy.engine.Engine COMMIT


### Read

**Option 1:** Default print output

In [53]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT name, rating FROM supplier"))
    for row in result.fetchall():
        print(row)
    conn.commit()

2025-12-08 09:25:39,580 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-08 09:25:39,584 INFO sqlalchemy.engine.Engine SELECT name, rating FROM supplier
2025-12-08 09:25:39,586 INFO sqlalchemy.engine.Engine [generated in 0.01781s] ()
('Ruiru II Farm', 3)
('Narok 41 Ranch', 4.8)
('Ngong Green House', 2)
('Makueni Fruits', 4)
('Pwani Farm', 1.8)
2025-12-08 09:25:39,611 INFO sqlalchemy.engine.Engine COMMIT


**Option 2:** Customized print output

In [54]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT name, rating FROM supplier"))
    for row in result.fetchall():
        print(f"name: {row.name},  rating: {row.rating}")
    conn.commit()

2025-12-08 09:25:44,864 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-08 09:25:44,867 INFO sqlalchemy.engine.Engine SELECT name, rating FROM supplier
2025-12-08 09:25:44,868 INFO sqlalchemy.engine.Engine [cached since 5.3s ago] ()
name: Ruiru II Farm,  rating: 3
name: Narok 41 Ranch,  rating: 4.8
name: Ngong Green House,  rating: 2
name: Makueni Fruits,  rating: 4
name: Pwani Farm,  rating: 1.8
2025-12-08 09:25:44,872 INFO sqlalchemy.engine.Engine COMMIT


### Update

In [55]:
with engine.connect() as conn:
    conn.execute(
        text("UPDATE supplier SET rating = :rating WHERE name = :name"),
        {"name": "Makueni Fruits", "rating": 5}
    )
    conn.commit()

2025-12-08 09:26:04,120 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-08 09:26:04,125 INFO sqlalchemy.engine.Engine UPDATE supplier SET rating = ? WHERE name = ?
2025-12-08 09:26:04,126 INFO sqlalchemy.engine.Engine [generated in 0.00821s] (5, 'Makueni Fruits')
2025-12-08 09:26:04,137 INFO sqlalchemy.engine.Engine COMMIT


With multiple statements in one database transaction.

In [56]:
with engine.connect() as conn:
    conn.execute(
        text("UPDATE supplier SET rating = :rating WHERE name = :name"),
        {"name": "Makueni Fruits", "rating": 4.5}
    )
    conn.execute(
        text("UPDATE supplier SET rating = :rating WHERE name = :name"),
        {"name": "Narok 41 Ranch", "rating": 4.9}
    )
    result = conn.execute(
        text("SELECT name, rating FROM supplier WHERE name = :name"),
        {"name": "Makueni Fruits"}
    )
    for row in result.fetchall():
        print(row)
    conn.commit()

2025-12-08 09:27:18,218 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-08 09:27:18,220 INFO sqlalchemy.engine.Engine UPDATE supplier SET rating = ? WHERE name = ?
2025-12-08 09:27:18,221 INFO sqlalchemy.engine.Engine [cached since 74.1s ago] (4.5, 'Makueni Fruits')
2025-12-08 09:27:18,225 INFO sqlalchemy.engine.Engine UPDATE supplier SET rating = ? WHERE name = ?
2025-12-08 09:27:18,227 INFO sqlalchemy.engine.Engine [cached since 74.11s ago] (4.9, 'Narok 41 Ranch')
2025-12-08 09:27:18,228 INFO sqlalchemy.engine.Engine SELECT name, rating FROM supplier WHERE name = ?
2025-12-08 09:27:18,230 INFO sqlalchemy.engine.Engine [generated in 0.00207s] ('Makueni Fruits',)
('Makueni Fruits', 4.5)
2025-12-08 09:27:18,241 INFO sqlalchemy.engine.Engine COMMIT


### Delete

In [57]:
with engine.connect() as conn:
    conn.execute(
        text("DELETE FROM supplier WHERE name = :name"),
        {"name": "Pwani Farm"}
    )

    conn.commit()

2025-12-08 09:27:34,692 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-08 09:27:34,695 INFO sqlalchemy.engine.Engine DELETE FROM supplier WHERE name = ?
2025-12-08 09:27:34,696 INFO sqlalchemy.engine.Engine [generated in 0.00428s] ('Pwani Farm',)
2025-12-08 09:27:34,699 INFO sqlalchemy.engine.Engine COMMIT


### Rollback to a Savepoint
- A savepoint allows you to set a point within a transaction that you can roll back to without affecting the entire transaction. This is useful when you want to undo certain operations while keeping others intact.

In [58]:
with engine.connect() as conn:
    trans = conn.begin()  # start transaction
    try:
        conn.execute(
            text("UPDATE supplier SET rating = :rating WHERE name = :name"),
            {"name": "Makueni Fruits", "rating": 4.3}
        )
        # Create a savepoint
        savepoint_1 = conn.begin_nested()
        conn.execute(
            text("UPDATE supplier SET rating = :rating WHERE name = :name"),
            {"name": "Narok 41 Ranch", "rating": 1.2}
        )
        # Rollback to savepoint (undo last update only)
        savepoint_1.rollback()
        result = conn.execute(
            text("SELECT name, rating FROM supplier WHERE name = :name"),
            {"name": "Makueni Fruits"}
        )
        for row in result.fetchall():
            print(row)
        trans.commit()
    except sqlalchemy.exc.SQLAlchemyError:
        trans.rollback()
        print("Transaction rolled back")

2025-12-08 09:29:47,940 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-08 09:29:47,941 INFO sqlalchemy.engine.Engine UPDATE supplier SET rating = ? WHERE name = ?
2025-12-08 09:29:47,942 INFO sqlalchemy.engine.Engine [cached since 223.8s ago] (4.3, 'Makueni Fruits')
2025-12-08 09:29:47,953 INFO sqlalchemy.engine.Engine SAVEPOINT sa_savepoint_1
2025-12-08 09:29:47,956 INFO sqlalchemy.engine.Engine [no key 0.00275s] ()
2025-12-08 09:29:47,957 INFO sqlalchemy.engine.Engine UPDATE supplier SET rating = ? WHERE name = ?
2025-12-08 09:29:47,958 INFO sqlalchemy.engine.Engine [cached since 223.8s ago] (1.2, 'Narok 41 Ranch')
2025-12-08 09:29:47,963 INFO sqlalchemy.engine.Engine ROLLBACK TO SAVEPOINT sa_savepoint_1
2025-12-08 09:29:47,965 INFO sqlalchemy.engine.Engine [no key 0.00176s] ()
2025-12-08 09:29:47,967 INFO sqlalchemy.engine.Engine SELECT name, rating FROM supplier WHERE name = ?
2025-12-08 09:29:47,968 INFO sqlalchemy.engine.Engine [cached since 149.7s ago] ('Makueni Fruits',

In [59]:
with engine.connect() as conn:
    trans = conn.begin()
    try:
        conn.execute(
            text("UPDATE supplier SET rating = :rating WHERE name = :name"),
            {"name": "Narok 41 Ranch", "rating": 0.5}
        )

        sp1 = conn.begin_nested()  # Savepoint 1
        conn.execute(
            text("UPDATE supplier SET rating = :rating WHERE name = :name"),
            {"name": "Ngong Green House", "rating": 0.9}
        )

        sp2 = conn.begin_nested()  # Savepoint 2
        conn.execute(
            text("UPDATE supplier SET rating = :rating WHERE name = :name"),
            {"name": "Narok 41 Ranch", "rating": 5.0}
        )
        sp2.rollback()  # Undo the last update only
        sp1.rollback()  # Undo up to Savepoint 1 (including changes after sp1)
        trans.commit()
    except sqlalchemy.exc.SQLAlchemyError:
        trans.rollback()

2025-12-08 09:30:57,957 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-08 09:30:57,959 INFO sqlalchemy.engine.Engine UPDATE supplier SET rating = ? WHERE name = ?
2025-12-08 09:30:57,961 INFO sqlalchemy.engine.Engine [cached since 293.8s ago] (0.5, 'Narok 41 Ranch')
2025-12-08 09:30:57,965 INFO sqlalchemy.engine.Engine SAVEPOINT sa_savepoint_1
2025-12-08 09:30:57,967 INFO sqlalchemy.engine.Engine [no key 0.00153s] ()
2025-12-08 09:30:57,968 INFO sqlalchemy.engine.Engine UPDATE supplier SET rating = ? WHERE name = ?
2025-12-08 09:30:57,969 INFO sqlalchemy.engine.Engine [cached since 293.9s ago] (0.9, 'Ngong Green House')
2025-12-08 09:30:57,971 INFO sqlalchemy.engine.Engine SAVEPOINT sa_savepoint_2
2025-12-08 09:30:57,972 INFO sqlalchemy.engine.Engine [no key 0.00114s] ()
2025-12-08 09:30:57,973 INFO sqlalchemy.engine.Engine UPDATE supplier SET rating = ? WHERE name = ?
2025-12-08 09:30:57,974 INFO sqlalchemy.engine.Engine [cached since 293.9s ago] (5.0, 'Narok 41 Ranch')
2025-1

## Close the Connection
- It is good practice to close the connection when you are done. This also commits anything that has not yet been committed up to this point.

In [8]:
conn.close()

## References
SQLAlchemy Project. (2025, December 5). _SQLAlchemy 2.0 Documentation._ SQLAlchemy. https://docs.sqlalchemy.org/en/20/
